The general strategy is to:

- clean the data is so it's in a more workable format
- merge all transportation and weather data into a single table
- create a decision function that labels all the days with a decision of whether to take the bus, train, walk or stay home
- train a model!
- Profit!

### Cleaning dataset

In [37]:
import pandas as pd

# Load the CSV files into pandas dataframes
weather_df = pd.read_csv('./datasets/nyc-weather.csv')
street_flooding_df = pd.read_csv('./datasets/street-flooding.csv')
bus_breakdowns_df = pd.read_csv('./datasets/bus-breakdowns-and-delays.csv')
motor_vehicle_collisions_df = pd.read_csv('./datasets/nyc-motor-vehicle-collisions.csv')
mta_traffic = pd.read_csv('./datasets/mta-traffic.csv')

print('Dataframes loaded successfully.')

/var/folders/qm/mct6b9p12yx1g2ky7vxrqws40000gn/T/ipykernel_54943/2502882040.py:5: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  street_flooding_df = pd.read_csv('./datasets/street-flooding.csv')
/var/folders/qm/mct6b9p12yx1g2ky7vxrqws40000gn/T/ipykernel_54943/2502882040.py:6: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  bus_breakdowns_df = pd.read_csv('./datasets/bus-breakdowns-and-delays.csv')
/var/folders/qm/mct6b9p12yx1g2ky7vxrqws40000gn/T/ipykernel_54943/2502882040.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  motor_vehicle_collisions_df = pd.read_csv('./datasets/nyc-motor-vehicle-collisions.csv')


Dataframes loaded successfully.


In [38]:
# Check the head of each dataframe to understand their structure and identify the primary key for joining
dfs = [weather_df, street_flooding_df, bus_breakdowns_df, motor_vehicle_collisions_df, mta_traffic]
df_names = ['Weather', 'Street Flooding', 'Bus Breakdowns', 'Motor Vehicle Collisions', 'MTA Traffic']

weather_df


,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2016-01-01T00:00,7.6,0.0,0.0,69.0,53.0,0.0,72.0,10.0,296.0
1,2016-01-01T01:00,7.5,0.0,0.0,20.0,4.0,0.0,56.0,9.8,287.0
2,2016-01-01T02:00,7.1,0.0,0.0,32.0,3.0,0.0,99.0,9.7,285.0
3,2016-01-01T03:00,6.6,0.0,0.0,35.0,5.0,0.0,100.0,9.2,281.0
4,2016-01-01T04:00,6.3,0.0,0.0,34.0,4.0,0.0,100.0,9.1,279.0
...,...,...,...,...,...,...,...,...,...,...
59755,2022-10-25T19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59756,2022-10-25T20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59757,2022-10-25T21:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59758,2022-10-25T22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
street_flooding_df

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,16843163,06/10/2010 04:52:00 PM,06/11/2010 08:30:00 AM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11415.0,124-22 QUEENS BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.712742,-73.827214,"(40.71274238328885, -73.8272142051277)"
1,16774888,06/01/2010 01:23:00 PM,06/02/2010 02:20:00 PM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11385.0,64-10 WOODBINE STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.708389,-73.895039,"(40.70838866371141, -73.89503901431905)"
2,16930799,06/23/2010 05:29:00 PM,06/25/2010 09:00:00 AM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11355.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.752248,-73.816623,"(40.75224780480794, -73.81662314501642)"
3,16922006,06/22/2010 10:06:00 PM,06/23/2010 09:40:00 AM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11234.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.627606,-73.913528,"(40.627605999420325, -73.91352821699182)"
4,17618462,07/09/2010 12:47:00 PM,07/10/2010 04:00:00 PM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,10023.0,2000 BROADWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.775607,-73.982149,"(40.775607416026595, -73.98214945684576)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39238,16777007,06/01/2010 08:23:00 AM,06/01/2010 11:00:00 AM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11355.0,MAIN STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.757856,-73.829303,"(40.75785560519996, -73.82930264121909)"
39239,16799206,06/04/2010 05:36:00 AM,06/04/2010 11:05:00 AM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11237.0,132 GRATTAN STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706278,-73.927803,"(40.70627844871178, -73.92780284272607)"
39240,16818740,06/07/2010 02:20:00 PM,06/08/2010 10:30:00 AM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,10306.0,157 MALONE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.559121,-74.124293,"(40.559121466195506, -74.1242934270326)"
39241,16872640,06/15/2010 03:56:00 PM,06/15/2010 06:55:00 PM,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),NaN,11212.0,419 EAST 98 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.658140,-73.915273,"(40.65814030145154, -73.91527271179912)"


In [40]:
bus_breakdowns_df

,School_Year,Busbreakdown_ID,Run_Type,Bus_No,Route_Number,Reason,Schools_Serviced,Occurred_On,Created_On,Boro,...,How_Long_Delayed,Number_Of_Students_On_The_Bus,Has_Contractor_Notified_Schools,Has_Contractor_Notified_Parents,Have_You_Alerted_OPT,Informed_On,Incident_Number,Last_Updated_On,Breakdown_or_Running_Late,School_Age_or_PreK
0,2015-2016,1227538,Special Ed AM Run,2621,J711,Heavy Traffic,75003,11/05/2015 08:10:00 AM,11/05/2015 08:12:00 AM,New Jersey,...,NaN,11,Yes,No,Yes,11/05/2015 08:12:00 AM,NaN,11/05/2015 08:12:14 AM,Running Late,School-Age
1,2015-2016,1227539,Special Ed AM Run,1260,M351,Heavy Traffic,06716,11/05/2015 08:10:00 AM,11/05/2015 08:12:00 AM,Manhattan,...,20MNS,2,Yes,Yes,No,11/05/2015 08:12:00 AM,NaN,11/05/2015 08:13:34 AM,Running Late,School-Age
2,2015-2016,1227540,Pre-K/EI,418,3,Heavy Traffic,C445,11/05/2015 08:09:00 AM,11/05/2015 08:13:00 AM,Bronx,...,15MIN,8,Yes,Yes,Yes,11/05/2015 08:13:00 AM,NaN,11/05/2015 08:13:22 AM,Running Late,Pre-K
3,2015-2016,1227541,Special Ed AM Run,4522,M271,Heavy Traffic,02699,11/05/2015 08:12:00 AM,11/05/2015 08:14:00 AM,Manhattan,...,15 MIN,6,No,No,No,11/05/2015 08:14:00 AM,NaN,11/05/2015 08:14:04 AM,Running Late,School-Age
4,2015-2016,1227542,Special Ed AM Run,3124,M373,Heavy Traffic,02116,11/05/2015 08:13:00 AM,11/05/2015 08:14:00 AM,Manhattan,...,NaN,6,No,No,No,11/05/2015 08:14:00 AM,NaN,11/05/2015 08:14:08 AM,Running Late,School-Age
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663838,2022-2023,1739299,Pre-K/EI,6121,B0505Z,Other,S272,09/13/2022 02:00:00 PM,09/13/2022 02:50:00 PM,Brooklyn,...,16-30 Min,0,Yes,Yes,No,09/13/2022 02:50:00 PM,NaN,09/13/2022 06:38:10 PM,Running Late,Pre-K
663839,2022-2023,1802544,Special Ed AM Run,24662,X081,Heavy Traffic,"10017,10095",03/08/2023 05:55:00 AM,03/08/2023 06:04:00 AM,Bronx,...,0-15 Min,0,Yes,Yes,No,03/08/2023 06:04:00 AM,NaN,01/01/1900 12:00:00 AM,Running Late,School-Age
663840,2022-2023,1802545,Special Ed AM Run,1959,N159,Heavy Traffic,02968,03/08/2023 06:03:00 AM,03/08/2023 06:04:00 AM,Manhattan,...,46-60 Min,0,Yes,Yes,Yes,03/08/2023 06:04:00 AM,NaN,01/01/1900 12:00:00 AM,Running Late,School-Age
663841,2022-2023,1802546,Special Ed AM Run,GT1479,Y127,Mechanical Problem,"08138,08420,08468",03/08/2023 06:00:00 AM,03/08/2023 06:04:00 AM,Bronx,...,0-15 Min,0,Yes,Yes,No,03/08/2023 06:04:00 AM,NaN,01/01/1900 12:00:00 AM,Running Late,School-Age


In [41]:
motor_vehicle_collisions_df

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2076483,03/05/2024,17:22,QUEENS,11436.0,40.680477,-73.792100,"(40.680477, -73.7921)",SUTPHIN BOULEVARD,FOCH BOULEVARD,NaN,...,Unspecified,NaN,NaN,NaN,4707511,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2076484,03/05/2024,17:00,BROOKLYN,11204.0,40.610786,-73.978820,"(40.610786, -73.97882)",NaN,NaN,161 AVENUE O,...,Unspecified,Unspecified,Unspecified,NaN,4707419,Ambulance,PK,Van,PK,NaN
2076485,03/03/2024,17:50,NaN,NaN,40.675053,-73.947235,"(40.675053, -73.947235)",SAINT MARKS AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4707855,Station Wagon/Sport Utility Vehicle,PK,NaN,NaN,NaN
2076486,03/05/2024,14:30,BROOKLYN,11207.0,40.677900,-73.892586,"(40.6779, -73.892586)",MILLER AVENUE,FULTON STREET,NaN,...,NaN,NaN,NaN,NaN,4707872,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


In [42]:
# Normalize the date formats across datasets for joining
weather_df['time'] = pd.to_datetime(weather_df['time']).dt.normalize()
street_flooding_df['Created Date'] = pd.to_datetime(street_flooding_df['Created Date']).dt.normalize()
bus_breakdowns_df['Occurred_On'] = pd.to_datetime(bus_breakdowns_df['Occurred_On']).dt.normalize()
motor_vehicle_collisions_df['CRASH DATE'] = pd.to_datetime(motor_vehicle_collisions_df['CRASH DATE']).dt.normalize()
mta_traffic['Date'] = pd.to_datetime(mta_traffic['Date']).dt.normalize()

print('Date formats normalized.')

/var/folders/qm/mct6b9p12yx1g2ky7vxrqws40000gn/T/ipykernel_54943/2505189228.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  street_flooding_df['Created Date'] = pd.to_datetime(street_flooding_df['Created Date']).dt.normalize()


Date formats normalized.


In [43]:
# Merge the datasets on the date columns
merged_df = weather_df.copy()
merged_df["flooding"] = merged_df["time"].map(
    street_flooding_df["Created Date"].value_counts()
)
merged_df["bus_breakdowns"] = merged_df["time"].map(
    bus_breakdowns_df["Occurred_On"].value_counts()
)
merged_df["motor_vehicle_collisions"] = merged_df["time"].map(
    motor_vehicle_collisions_df["CRASH DATE"].value_counts()
)
merged_df["mta_traffic"] = merged_df["time"].map(mta_traffic.groupby("Date")["# Vehicles - VToll"].sum())


# Fill NaN values with 0
merged_df[["flooding", "bus_breakdowns", "motor_vehicle_collisions", "mta_traffic"]] = (
    merged_df[
        ["flooding", "bus_breakdowns", "motor_vehicle_collisions", "mta_traffic"]
    ].fillna(0)
)

# Convert the counts to integers
merged_df[["flooding", "bus_breakdowns", "motor_vehicle_collisions", "mta_traffic"]] = (
    merged_df[
        ["flooding", "bus_breakdowns", "motor_vehicle_collisions", "mta_traffic"]
    ].astype(int)
)

merged_df

,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°),flooding,bus_breakdowns,motor_vehicle_collisions,mta_traffic
0,2016-01-01,7.6,0.0,0.0,69.0,53.0,0.0,72.0,10.0,296.0,0,0,393,109881
1,2016-01-01,7.5,0.0,0.0,20.0,4.0,0.0,56.0,9.8,287.0,0,0,393,109881
2,2016-01-01,7.1,0.0,0.0,32.0,3.0,0.0,99.0,9.7,285.0,0,0,393,109881
3,2016-01-01,6.6,0.0,0.0,35.0,5.0,0.0,100.0,9.2,281.0,0,0,393,109881
4,2016-01-01,6.3,0.0,0.0,34.0,4.0,0.0,100.0,9.1,279.0,0,0,393,109881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59755,2022-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,845,295,115758
59756,2022-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,845,295,115758
59757,2022-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,845,295,115758
59758,2022-10-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,845,295,115758


In [44]:

# drop na
merged_df = merged_df.dropna()

merged_df

,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°),flooding,bus_breakdowns,motor_vehicle_collisions,mta_traffic
0,2016-01-01,7.6,0.0,0.0,69.0,53.0,0.0,72.0,10.0,296.0,0,0,393,109881
1,2016-01-01,7.5,0.0,0.0,20.0,4.0,0.0,56.0,9.8,287.0,0,0,393,109881
2,2016-01-01,7.1,0.0,0.0,32.0,3.0,0.0,99.0,9.7,285.0,0,0,393,109881
3,2016-01-01,6.6,0.0,0.0,35.0,5.0,0.0,100.0,9.2,281.0,0,0,393,109881
4,2016-01-01,6.3,0.0,0.0,34.0,4.0,0.0,100.0,9.1,279.0,0,0,393,109881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59611,2022-10-19,12.2,0.0,0.0,68.0,9.0,100.0,0.0,21.2,212.0,5,746,286,119790
59612,2022-10-19,12.2,0.0,0.0,61.0,8.0,90.0,0.0,20.3,207.0,5,746,286,119790
59613,2022-10-19,12.2,0.0,0.0,62.0,21.0,72.0,0.0,16.6,207.0,5,746,286,119790
59614,2022-10-19,11.1,0.0,0.0,4.0,0.0,6.0,0.0,10.8,206.0,5,746,286,119790


In [47]:
# Calculate the 75th percentile for each column
mta_traffic_75 = merged_df['mta_traffic'].quantile(0.75)
bus_breakdowns_75 = merged_df['bus_breakdowns'].quantile(0.75)
motor_vehicle_collisions_75 = merged_df['motor_vehicle_collisions'].quantile(0.75)
rain_75 = merged_df['rain (mm)'].quantile(0.75)

# Define the decision function
def decision(row):
    options = ['subway', 'bus', 'car', 'walk', 'stay_home']
    if row['mta_traffic'] > mta_traffic_75:
        options.remove('subway')
    if row['bus_breakdowns'] > bus_breakdowns_75:
        
        options.remove('bus')
    if row['motor_vehicle_collisions'] > motor_vehicle_collisions_75:
        options.remove('car')
    if row['rain (mm)'] > rain_75:
        options.remove('walk')

    return options[0]

# Apply the decision function to each row
merged_df.loc[:, 'decision'] = merged_df.apply(decision, axis=1)

merged_df

,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°),flooding,bus_breakdowns,motor_vehicle_collisions,mta_traffic,decision
0,2016-01-01,7.6,0.0,0.0,69.0,53.0,0.0,72.0,10.0,296.0,0,0,393,109881,subway
1,2016-01-01,7.5,0.0,0.0,20.0,4.0,0.0,56.0,9.8,287.0,0,0,393,109881,subway
2,2016-01-01,7.1,0.0,0.0,32.0,3.0,0.0,99.0,9.7,285.0,0,0,393,109881,subway
3,2016-01-01,6.6,0.0,0.0,35.0,5.0,0.0,100.0,9.2,281.0,0,0,393,109881,subway
4,2016-01-01,6.3,0.0,0.0,34.0,4.0,0.0,100.0,9.1,279.0,0,0,393,109881,subway
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59611,2022-10-19,12.2,0.0,0.0,68.0,9.0,100.0,0.0,21.2,212.0,5,746,286,119790,car
59612,2022-10-19,12.2,0.0,0.0,61.0,8.0,90.0,0.0,20.3,207.0,5,746,286,119790,car
59613,2022-10-19,12.2,0.0,0.0,62.0,21.0,72.0,0.0,16.6,207.0,5,746,286,119790,car
59614,2022-10-19,11.1,0.0,0.0,4.0,0.0,6.0,0.0,10.8,206.0,5,746,286,119790,car


In [48]:
# Rename the columns to simpler names in the pandas dataframe 'merged_df'
merged_df.columns = ['time', 'temp', 'precipitation', 'rain', 'cloudcover', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_high', 'windspeed', 'winddirection', 'flooding', 'bus_breakdowns', 'collisions', 'traffic', 'decision']

merged_df

,time,temp,precipitation,rain,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed,winddirection,flooding,bus_breakdowns,collisions,traffic,decision
0,2016-01-01,7.6,0.0,0.0,69.0,53.0,0.0,72.0,10.0,296.0,0,0,393,109881,subway
1,2016-01-01,7.5,0.0,0.0,20.0,4.0,0.0,56.0,9.8,287.0,0,0,393,109881,subway
2,2016-01-01,7.1,0.0,0.0,32.0,3.0,0.0,99.0,9.7,285.0,0,0,393,109881,subway
3,2016-01-01,6.6,0.0,0.0,35.0,5.0,0.0,100.0,9.2,281.0,0,0,393,109881,subway
4,2016-01-01,6.3,0.0,0.0,34.0,4.0,0.0,100.0,9.1,279.0,0,0,393,109881,subway
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59611,2022-10-19,12.2,0.0,0.0,68.0,9.0,100.0,0.0,21.2,212.0,5,746,286,119790,car
59612,2022-10-19,12.2,0.0,0.0,61.0,8.0,90.0,0.0,20.3,207.0,5,746,286,119790,car
59613,2022-10-19,12.2,0.0,0.0,62.0,21.0,72.0,0.0,16.6,207.0,5,746,286,119790,car
59614,2022-10-19,11.1,0.0,0.0,4.0,0.0,6.0,0.0,10.8,206.0,5,746,286,119790,car


In [49]:
from fastai.tabular.all import (
    TabularDataLoaders,
    Categorify,
    FillMissing,
    Normalize,
    range_of,
    RandomSplitter,
    accuracy,
    tabular_learner,
    ShowGraphCallback,
)

In [50]:
dls = TabularDataLoaders.from_df(
    df=merged_df, 
    y_names="decision",
    cat_names=['decision'],
    cont_names=['temp', 'precipitation', 'rain', 'cloudcover', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_high', 'windspeed', 'winddirection', 'flooding', 'bus_breakdowns', 'collisions', 'traffic'],
    procs = [Categorify, FillMissing, Normalize])

In [51]:
learn = tabular_learner(dls, metrics=accuracy)


In [52]:
learn.fit_one_cycle(1)


epoch,train_loss,valid_loss,accuracy,time
0,0.004300,0.002350,0.999664,00:10


In [61]:
row, clas, probs = learn.predict(merged_df.iloc[1200])

row.show()


,decision,decision,temp,precipitation,rain,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed,winddirection,flooding,bus_breakdowns,collisions,traffic,decision,decision
0,bus,bus,1.8,0.4,-1.126380e-09,56.0,2.0,40.0,100.000001,11.1,154.999999,-1.511521e-08,0.000004,554.999999,126471.000505,bus,bus
